# Ingest the Cleaned Plot into table

In [76]:
import pandas as pd
import re
import nltk
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine

# nltk resources are download
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\84146\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\84146\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Connection to the Database

In [77]:
import configparser

def read_config():
    config = configparser.ConfigParser()
    config.read('config.ini')
    return {
        'serveradminlogin': config['SQL']['serveradminlogin'],
        'serveradminpassword': config['SQL']['serveradminpassword'],
        'database': config['SQL']['database'],
        'servername': config['SQL']['servername']
    }


serveradminlogin = read_config()['serveradminlogin']
serveradminpassword = read_config()['serveradminpassword']
database = read_config()['database']
servername = read_config()['servername']

## Ingesting the data into the table in SQL

In [78]:
file_path = '../cleaned_movies1.csv'
df = pd.read_csv(file_path)

df.head()

,imdbID,title,year,rating,runtime,genre,released,director,writer,cast,...,imdbRating,imdbVotes,poster,plot,fullplot,language,country,awards,lastupdated,type
0,1,Carmencita,1894,NOT RATED,1 min,"Documentary, Short",NaN,William K.L. Dickson,NaN,Carmencita,...,5.9,1032.0,https://m.media-amazon.com/images/M/MV5BMjAzND...,Performing on what looks like a small wooden s...,Performing on what looks like a small wooden s...,NaN,USA,NaN,2015-08-26 00:03:45.040000000,movie
1,230,Cinderella,1899,NaN,6 min,"Drama, Short",1899-12-25,Georges M�li�s,Charles Perrault (story),"Barral, Bleuette Bernon, Carmely, Jeanne d'Alcy",...,6.6,586.0,https://m.media-amazon.com/images/M/MV5BMTgwMD...,A fairy godmother magically turns Cinderella's...,A fairy godmother magically turns Cinderella's...,NaN,France,NaN,2015-08-29 00:20:56.217000000,movie
2,399,Jack and the Beanstalk,1902,NaN,10 min,"Short, Fantasy",1902-07-15,"George S. Fleming, Edwin S. Porter",NaN,Thomas White,...,6.2,442.0,https://m.media-amazon.com/images/M/MV5BMjAzNT...,Porter's sequential continuity editing links s...,Porter's sequential continuity editing links s...,English,USA,NaN,2015-08-29 00:25:25.360000000,movie
3,417,A Trip to the Moon,1902,TV-G,13 min,"Short, Adventure, Fantasy",1902-10-04,Georges M�li�s,NaN,"Fran�ois Lallement, Jules-Eug�ne Legris",...,8.2,23904.0,https://m.media-amazon.com/images/M/MV5BMTQzMD...,A group of astronomers go on an expedition to ...,A group of men travel to the moon by being sho...,NaN,France,NaN,2015-09-01 00:16:55.443000000,movie
4,447,Life of an American Fireman,1903,NOT RATED,6 min,"Short, Action",1903-01-01,"George S. Fleming, Edwin S. Porter",NaN,"Vivian Vaughan, Arthur White, James H. White",...,6.4,1158.0,https://m.media-amazon.com/images/M/MV5BMjAzNT...,Porter's sequential continuity editing links s...,Porter's sequential continuity editing links s...,English,USA,NaN,2015-08-29 00:26:36.093000000,movie


In [79]:
df['clean_plot'] = df['plot'].str.lower() #Convert to lower case

#Regex, numbers and spaces and puncuations

df['clean_plot'] = df['clean_plot'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
df['clean_plot'] = df['clean_plot'].apply(lambda x: re.sub('\s+', ' ', x))
df['clean_plot']

# tokenize the sentence

# Convert all entries in 'clean_plot' to strings (or handle NaNs differently if preferred)
# df['clean_plot'] = df['clean_plot'].astype(str)

# Now apply the tokenization
df['clean_plot'] = df['clean_plot'].apply(lambda x: nltk.word_tokenize(x))

df['clean_plot']

0        [performing, on, what, looks, like, a, small, ...
1        [a, fairy, godmother, magically, turns, cinder...
2        [porter, s, sequential, continuity, editing, l...
3        [a, group, of, astronomers, go, on, an, expedi...
4        [porter, s, sequential, continuity, editing, l...
                               ...                        
34143    [arianna, is, nineteen, years, old, and, still...
34144    [five, interlocking, tales, of, terror, follow...
34145    [a, squad, of, unsuspecting, cops, goes, throu...
34146    [on, a, desolate, island, suicidal, sheep, fra...
34147    [the, story, of, a, minibus, driver, and, a, f...
Name: clean_plot, Length: 34148, dtype: object

In [80]:
# Remove all the stop words 
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
plot = []
for sentence in df['clean_plot']:
    temp = []
    for word in sentence:
        if word not in stop_words and len(word) >= 3:
            temp.append(word)
    plot.append(temp)
plot

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\84146\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['performing',
  'looks',
  'like',
  'small',
  'wooden',
  'stage',
  'wearing',
  'dress',
  'hoop',
  'skirt',
  'white',
  'high',
  'heeled',
  'pumps',
  'carmencita',
  'dance',
  'kicks',
  'twirls',
  'smile',
  'always',
  'face'],
 ['fairy',
  'godmother',
  'magically',
  'turns',
  'cinderella',
  'rags',
  'beautiful',
  'dress',
  'pumpkin',
  'coach',
  'cinderella',
  'goes',
  'ball',
  'meets',
  'prince',
  'remember',
  'leave',
  'magic',
  'runs'],
 ['porter',
  'sequential',
  'continuity',
  'editing',
  'links',
  'several',
  'shots',
  'form',
  'narrative',
  'famous',
  'fairy',
  'tale',
  'story',
  'jack',
  'magic',
  'beanstalk',
  'borrowing',
  'cinematographic',
  'methods'],
 ['group', 'astronomers', 'expedition', 'moon'],
 ['porter',
  'sequential',
  'continuity',
  'editing',
  'links',
  'several',
  'shots',
  'form',
  'narrative',
  'firemen',
  'responding',
  'house',
  'fire',
  'leave',
  'station',
  'horse',
  'drawn',
  'pumper',
 

In [81]:
# Assign plot to the df
df['clean_plot'] = plot

In [82]:
df['clean_plot']

0        [performing, looks, like, small, wooden, stage...
1        [fairy, godmother, magically, turns, cinderell...
2        [porter, sequential, continuity, editing, link...
3                   [group, astronomers, expedition, moon]
4        [porter, sequential, continuity, editing, link...
                               ...                        
34143    [arianna, nineteen, years, old, still, first, ...
34144    [five, interlocking, tales, terror, follow, fa...
34145    [squad, unsuspecting, cops, goes, trapdoor, he...
34146    [desolate, island, suicidal, sheep, franck, me...
34147    [story, minibus, driver, fairytale, princesss,...
Name: clean_plot, Length: 34148, dtype: object

In [83]:
df.head()

,imdbID,title,year,rating,runtime,genre,released,director,writer,cast,...,imdbVotes,poster,plot,fullplot,language,country,awards,lastupdated,type,clean_plot
0,1,Carmencita,1894,NOT RATED,1 min,"Documentary, Short",NaN,William K.L. Dickson,NaN,Carmencita,...,1032.0,https://m.media-amazon.com/images/M/MV5BMjAzND...,Performing on what looks like a small wooden s...,Performing on what looks like a small wooden s...,NaN,USA,NaN,2015-08-26 00:03:45.040000000,movie,"[performing, looks, like, small, wooden, stage..."
1,230,Cinderella,1899,NaN,6 min,"Drama, Short",1899-12-25,Georges M�li�s,Charles Perrault (story),"Barral, Bleuette Bernon, Carmely, Jeanne d'Alcy",...,586.0,https://m.media-amazon.com/images/M/MV5BMTgwMD...,A fairy godmother magically turns Cinderella's...,A fairy godmother magically turns Cinderella's...,NaN,France,NaN,2015-08-29 00:20:56.217000000,movie,"[fairy, godmother, magically, turns, cinderell..."
2,399,Jack and the Beanstalk,1902,NaN,10 min,"Short, Fantasy",1902-07-15,"George S. Fleming, Edwin S. Porter",NaN,Thomas White,...,442.0,https://m.media-amazon.com/images/M/MV5BMjAzNT...,Porter's sequential continuity editing links s...,Porter's sequential continuity editing links s...,English,USA,NaN,2015-08-29 00:25:25.360000000,movie,"[porter, sequential, continuity, editing, link..."
3,417,A Trip to the Moon,1902,TV-G,13 min,"Short, Adventure, Fantasy",1902-10-04,Georges M�li�s,NaN,"Fran�ois Lallement, Jules-Eug�ne Legris",...,23904.0,https://m.media-amazon.com/images/M/MV5BMTQzMD...,A group of astronomers go on an expedition to ...,A group of men travel to the moon by being sho...,NaN,France,NaN,2015-09-01 00:16:55.443000000,movie,"[group, astronomers, expedition, moon]"
4,447,Life of an American Fireman,1903,NOT RATED,6 min,"Short, Action",1903-01-01,"George S. Fleming, Edwin S. Porter",NaN,"Vivian Vaughan, Arthur White, James H. White",...,1158.0,https://m.media-amazon.com/images/M/MV5BMjAzNT...,Porter's sequential continuity editing links s...,Porter's sequential continuity editing links s...,English,USA,NaN,2015-08-29 00:26:36.093000000,movie,"[porter, sequential, continuity, editing, link..."


In [84]:
df['genre']

0               Documentary, Short
1                     Drama, Short
2                   Short, Fantasy
3        Short, Adventure, Fantasy
4                    Short, Action
                   ...            
34143                        Drama
34144                       Horror
34145              Fantasy, Horror
34146             Animation, Short
34147     Comedy, Fantasy, Romance
Name: genre, Length: 34148, dtype: object

In [85]:
df['genre'] = df['genre'].apply(lambda x: x.split(','))
df['cast'] = df['cast'].apply(lambda x: x.split(',')[:4]) # Getting the top 4 actors
df['director'] = df['director'].apply(lambda x: x.split(',')[:1]) # Getting 1 director


In [86]:
df['cast'][0]

['Carmencita']

In [87]:
def clean(sentence):
    temp = []
    for word in sentence:
        temp.append(word.lower().replace(' ', ''))
    return temp
# to clean the sentence all lower and remove all spaces

In [88]:
df['genre'] = [clean(x) for x in df['genre']]
df['cast'] = [clean(x) for x in df['cast']]
df['director'] = [clean(x) for x in df['director']]

In [89]:
df['cast'][0]

['carmencita']

In [90]:
df['genre'][0]

['documentary', 'short']

In [91]:
#combine all of the preproccesing into another dataframe

columns = ['clean_plot', 'genre', 'cast', 'director']
l = []

for i in range(len(df)):
    words = ''
    for col in columns:
        words += ' '.join(df[col][i]) + ' '
    l.append(words)

l

['performing looks like small wooden stage wearing dress hoop skirt white high heeled pumps carmencita dance kicks twirls smile always face documentary short carmencita williamk.l.dickson ',
 "fairy godmother magically turns cinderella rags beautiful dress pumpkin coach cinderella goes ball meets prince remember leave magic runs drama short barral bleuettebernon carmely jeanned'alcy georgesm�li�s ",
 'porter sequential continuity editing links several shots form narrative famous fairy tale story jack magic beanstalk borrowing cinematographic methods short fantasy thomaswhite georges.fleming ',
 'group astronomers expedition moon short adventure fantasy fran�oislallement jules-eug�nelegris georgesm�li�s ',
 'porter sequential continuity editing links several shots form narrative firemen responding house fire leave station horse drawn pumper arrive short action vivianvaughan arthurwhite jamesh.white georges.fleming ',
 "group bandits stage brazen train hold find determined posse hot heel

In [92]:
df.head()

,imdbID,title,year,rating,runtime,genre,released,director,writer,cast,...,imdbVotes,poster,plot,fullplot,language,country,awards,lastupdated,type,clean_plot
0,1,Carmencita,1894,NOT RATED,1 min,"[documentary, short]",NaN,[williamk.l.dickson],NaN,[carmencita],...,1032.0,https://m.media-amazon.com/images/M/MV5BMjAzND...,Performing on what looks like a small wooden s...,Performing on what looks like a small wooden s...,NaN,USA,NaN,2015-08-26 00:03:45.040000000,movie,"[performing, looks, like, small, wooden, stage..."
1,230,Cinderella,1899,NaN,6 min,"[drama, short]",1899-12-25,[georgesm�li�s],Charles Perrault (story),"[barral, bleuettebernon, carmely, jeanned'alcy]",...,586.0,https://m.media-amazon.com/images/M/MV5BMTgwMD...,A fairy godmother magically turns Cinderella's...,A fairy godmother magically turns Cinderella's...,NaN,France,NaN,2015-08-29 00:20:56.217000000,movie,"[fairy, godmother, magically, turns, cinderell..."
2,399,Jack and the Beanstalk,1902,NaN,10 min,"[short, fantasy]",1902-07-15,[georges.fleming],NaN,[thomaswhite],...,442.0,https://m.media-amazon.com/images/M/MV5BMjAzNT...,Porter's sequential continuity editing links s...,Porter's sequential continuity editing links s...,English,USA,NaN,2015-08-29 00:25:25.360000000,movie,"[porter, sequential, continuity, editing, link..."
3,417,A Trip to the Moon,1902,TV-G,13 min,"[short, adventure, fantasy]",1902-10-04,[georgesm�li�s],NaN,"[fran�oislallement, jules-eug�nelegris]",...,23904.0,https://m.media-amazon.com/images/M/MV5BMTQzMD...,A group of astronomers go on an expedition to ...,A group of men travel to the moon by being sho...,NaN,France,NaN,2015-09-01 00:16:55.443000000,movie,"[group, astronomers, expedition, moon]"
4,447,Life of an American Fireman,1903,NOT RATED,6 min,"[short, action]",1903-01-01,[georges.fleming],NaN,"[vivianvaughan, arthurwhite, jamesh.white]",...,1158.0,https://m.media-amazon.com/images/M/MV5BMjAzNT...,Porter's sequential continuity editing links s...,Porter's sequential continuity editing links s...,English,USA,NaN,2015-08-29 00:26:36.093000000,movie,"[porter, sequential, continuity, editing, link..."


In [93]:
df.columns

Index(['imdbID', 'title', 'year', 'rating', 'runtime', 'genre', 'released',
       'director', 'writer', 'cast', 'metacritic', 'imdbRating', 'imdbVotes',
       'poster', 'plot', 'fullplot', 'language', 'country', 'awards',
       'lastupdated', 'type', 'clean_plot'],
      dtype='object')

In [94]:
df['clean_input'] = l
df = df[['title', 'poster', 'clean_input']]
df.head() #Processed with the title with the input from genre, plot, actors and director

,title,poster,clean_input
0,Carmencita,https://m.media-amazon.com/images/M/MV5BMjAzND...,performing looks like small wooden stage weari...
1,Cinderella,https://m.media-amazon.com/images/M/MV5BMTgwMD...,fairy godmother magically turns cinderella rag...
2,Jack and the Beanstalk,https://m.media-amazon.com/images/M/MV5BMjAzNT...,porter sequential continuity editing links sev...
3,A Trip to the Moon,https://m.media-amazon.com/images/M/MV5BMTQzMD...,group astronomers expedition moon short advent...
4,Life of an American Fireman,https://m.media-amazon.com/images/M/MV5BMjAzNT...,porter sequential continuity editing links sev...


In [95]:
df['clean_input'][0]

'performing looks like small wooden stage wearing dress hoop skirt white high heeled pumps carmencita dance kicks twirls smile always face documentary short carmencita williamk.l.dickson '

In [96]:
# Function to insert data into Azure SQL Database
def insert_into_sql(df, server, database, username, password):
    driver = 'ODBC Driver 17 for SQL Server'
    connection_string = f"mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver={driver}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)

    # Insert data into Azure SQL table
    df.to_sql('cleaned_data', con=engine, if_exists='replace', index=False)

# Insert data into Azure SQL
insert_into_sql(df, servername, database, serveradminlogin, serveradminpassword)